<a href="https://colab.research.google.com/github/santiago-vas/convolutional-neural-network-CNN-to-predict-stress-in-mechanical-displacemnets/blob/main/refere_BIELA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy import asarray
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Conv2DTranspose
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
import keras.backend as K
from keras import metrics
from numpy.random import RandomState
import scipy.io
from sklearn.externals import joblib

rng=RandomState(0)

w1 = scipy.io.loadmat('/content/drive/My Drive/Colab Notebooks/pca BIELA E/parte 1/B_peso1.mat')
w2 = scipy.io.loadmat('/content/drive/My Drive/Colab Notebooks/pca BIELA E/parte 1/B_100_peso2.mat')
out = scipy.io.loadmat('/content/drive/My Drive/Colab Notebooks/pca BIELA E/parte 1/B_100_salidaFinal.mat')
L = scipy.io.loadmat('/content/drive/My Drive/Colab Notebooks/pca BIELA E/parte 1/B_100_valoresLL2.mat')

L=L['uLL2']
ww1=w1['aW1']
ww2=w2['bW2']
ref = out['YF']

inEst = pd.read_excel('/content/drive/My Drive/Colab Notebooks/arreglar datos de la BIELA ESTANDAR/_in_var_PREF.xlsx')
inr=inEst.to_numpy()
print(inr.shape,ref.shape,ww1.shape,ww2.shape,L.shape)
# ,ref.shape,ww1.shape,ww2.shape

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


(90, 3) (1, 1, 150, 90) (8, 19, 2, 280) (4, 4, 280, 150) (4480, 1)


In [ ]:
def CreateModel_NonlinearMapping(Xshape, Yshape):
  model = Sequential()
  model.add(Dense(Yshape*2, input_shape=(Xshape,), kernel_initializer='normal', activation='softplus'))
  model.add(Dense(Yshape*2, kernel_initializer='normal', activation='softplus'))
  model.add(Dense(Yshape, kernel_initializer='normal', activation='linear'))
  # model.compile(loss='mean_squared_error', optimizer='adamax')
  model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adamax(learning_rate=0.0005))
  return model

def CreateModel_StressDecoding(W1_in, W2_in,entrada):
  model = Sequential()
  pca_valor2=280
  model.add(Conv2DTranspose(filters=pca_valor2, kernel_size=[4,4], strides=(1,1), input_shape=(1,1,entrada),data_format='channels_last'))
  model.add(Conv2DTranspose(filters=2, kernel_size=[8,19], strides=(8,19)))
  model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001))
  W0=model.layers[0].get_weights()
  W1=model.layers[1].get_weights()
  W0[0]=W2_in
  W1[0]=W1_in
  model.layers[0].set_weights(W0)
  model.layers[1].set_weights(W1)
  return model

In [ ]:
muestra=90
nodos=1860
entrada=3
salida=150
veces=50

In [ ]:
muestra=90
nodos=1860
entrada=3
salida=150
veces=50
inr2=inr
ref2=ref.reshape(-1,muestra)
ref2=ref2.transpose()
# print(inr2[0,:])
# print(ref2[0,:])
print(inr2.shape)
print(ref2.shape)

(90, 3)
(90, 150)


In [ ]:
muestra=90
nodos=1860
entrada=3
salida=150
veces=50
inr2=inr
ref2=ref.reshape(-1,muestra)
ref2=ref2.transpose()
# print(inr2[0,:])
# print(ref2[0,:])
print(inr2.shape)
print(ref2.shape)
#*********************************
qlpca=QuantileTransformer()
qpca=qlpca.fit_transform(ref2)
#**********************************

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_data.py:2357: UserWarning: n_quantiles (1000) is greater than the total number of samples (90). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))


In [ ]:
muestra=90
nodos=1860
entrada=3
salida=150
veces=50
inr2=inr
ref2=ref.reshape(-1,muestra)
ref2=ref2.transpose()
# print(inr2[0,:])
# print(ref2[0,:])
print(inr2.shape)
print(ref2.shape)
#*********************************
qlpca=QuantileTransformer()
qpca=qlpca.fit_transform(ref2)
#**********************************


model=CreateModel_NonlinearMapping(entrada,salida)

# model = keras.models.load_model('/content/drive/My Drive/Colab Notebooks/pca BIELA E/parte 1/B_red1_referencia.h5')
veces
loss=np.zeros(veces)

for k in range(veces):
  id=np.arange(0,muestra,1)
  rng.shuffle(id)
  id_train=id[0:int(muestra*0.9)]
  id_test=id[int(muestra*0.9):]

  rp1=inr2[id_train,:]
  rp2=inr2[id_test,:]

  # rx1=ref2[id_train,:]
  # rx2=ref2[id_test,:]
  rx1=qpca[id_train,:]
  rx2=qpca[id_test,:]

  model.fit(rp1,rx1, epochs=5000,batch_size=10, verbose=0)
  loss[k]=model.evaluate(rp2,rx2,verbose=0)
  print(k,loss[k])

ploss= pd.DataFrame(loss)
ploss.to_excel('/content/drive/My Drive/Colab Notebooks/pca BIELA E/parte 1/B_errorMSE_red1_v2.xlsx', index=False)
model.save('/content/drive/My Drive/Colab Notebooks/pca BIELA E/parte 1/B_red1_referencia_v2.h5')
joblib.dump(qlpca,'/content/drive/My Drive/Colab Notebooks/pca BIELA E/parte 1/B_qlpca_referencia.pkl')

0 0.004199352581053972
1 0.003893052926287055
2 0.006439791060984135
3 0.0020506626460701227
4 0.001875799149274826
5 0.01047238428145647
6 0.002410539425909519
7 0.005316359922289848
8 0.009172886610031128
9 0.003017590381205082
10 0.0018543095793575048
11 0.0018878084374591708
12 0.0023602203000336885
13 0.0036406053695827723
14 0.0034066748339682817
15 0.002367190085351467
16 0.002873549470677972
17 0.008777011185884476
18 0.014415547251701355
19 0.004390094429254532
20 0.018647832795977592
21 0.006238247267901897
22 0.0024469299241900444
23 0.002285931259393692
24 0.0006155946175567806
25 0.0012006741017103195
26 0.004284609109163284
27 0.0018583906348794699
28 0.004100462421774864
29 0.0044237105175852776
30 0.006001447327435017
31 0.003867226419970393
32 0.0018771191826090217
33 0.001537557109259069
34 0.019457314163446426
35 0.011977961286902428
36 0.001369425212033093
37 0.0007795029086992145
38 0.0009632774745114148
39 0.004981744568794966
40 0.008882922120392323
41 0.00121473

['/content/drive/My Drive/Colab Notebooks/pca BIELA E/parte 1/B_qlpca_referencia.pkl']

In [ ]:
a=0
print(inr[a:a+1,:])
print(ref2[a:a+1,:])
####### model = keras.models.load_model('/content/drive/My Drive/Colab Notebooks/pca RIEL2D/parte1/20 pca/red1_dense_PCA20.h5')

Yp=model.predict(inr[a:a+1,:])
Yp=qlpca.inverse_transform(Yp)
for n in range(0, salida):
  Yp[:,n]=Yp[:,n]*L[n]
Ypp=Yp.reshape(1,1,1,salida)
print(Ypp.shape)

Yp=model.predict(input)
Yp=qlpca.inverse_transform(Yp)
entrada_Encoder=salida
Aconv=CreateModel_StressDecoding(ww1,ww2,entrada_Encoder)
datos_equivalentes=decoder(ww1,ww2,entrada_Encoder)

Aconv.save('/content/drive/My Drive/Colab Notebooks/pca BIELA E/parte 1/B_referencia_CONVT.h5')
# puntos=Aconv.predict(Ypp)
# punx=puntos[:,:,:,0].reshape(1860,order='F')
# puny=puntos[:,:,:,1].reshape(1860,order='F')
# # print(punf.shape)
# plt.plot(punx,puny,'*')
# plt.show()